In [1]:
# Reference used: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
import nltk 
import re 
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import pandas as pd   
import jsonlines
import json
from tqdm.notebook import tqdm
import pickle

In [2]:
documents = []
docID = {}
term_total_citations = {}
author_term_citations = {}
idx = 0
with jsonlines.open('data_india_sample.jl') as reader:
    for obj in tqdm(reader, leave=False):
        documents.append(obj)
        docID[idx] = obj['user']
        author_term_citations[obj['user']] = {}
        idx += 1

0it [00:00, ?it/s]

In [3]:
# Code Taken From: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
def remove_string_special_characters(s):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', ' ', s)
#     print(stripped)
    stripped = re.sub('_', ' ', stripped)
      
    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)
      
    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
            return stripped.lower()

In [4]:
# Code Reference: https://www.geeksforgeeks.org/python-stemming-words-with-nltk/
def stem_string(sentence):
    ps = PorterStemmer()
    
    words = word_tokenize(sentence)
    words = [ps.stem(word) for word in words]
    return " ".join(words)

In [5]:
def remove_stop_words(sentence):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(sentence)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

In [6]:
def process_string(sentence):
    sentence = remove_string_special_characters(sentence)
    sentence = stem_string(sentence)
    sentence = remove_stop_words(sentence)
    return sentence

### TF-IDF Matrix

In [7]:
def update_term_citations(document, sentence, numCitations, userID):
    numCitations += 1
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence)
    tokens = [token for token in sentence.split(" ") if token != ""]
    for n in range(1, 5):
#         seen_ngms = []
        ngms = list(ngrams(tokens, n))
        for ngm in ngms:
            gm = " ".join(ngm)
#             if gm in seen_ngms:
#                 continue
#             seen_ngms.append(gm)
            if gm in author_term_citations[userID]:
                author_term_citations[userID][gm] += numCitations
            else:
                author_term_citations[userID][gm] = numCitations
            if gm in term_total_citations:
                term_total_citations[gm] += numCitations
            else:
                term_total_citations[gm] = numCitations

In [29]:
word_documents = []
for document in tqdm(documents, leave=False):
    sentences = []
    for subject in document['subjects']:
        sentence = process_string(subject)
        sentences.append(sentence)
        update_term_citations(document, sentence, int(document['citationsAll']), document['user'])
    for paper in document['papers']:
        try:
            sentence = paper['title'] + " " + paper['conference']
            sentence = process_string(sentence)
        except: 
            continue
#         print(sentence)
#         print(sentence)
        update_term_citations(document, sentence, paper['citations'], document['user'])
        sentences.append(sentence)

#     word_documents.append(sentences)
    word_documents.append(".\n".join(sentences))
#     print(word_documents[-1])
#     word_documents.append(sentences)
#     print(".".join(sentences))

  0%|          | 0/10 [00:00<?, ?it/s]

In [40]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
tfidf_matrix = vectorizer.fit_transform(word_documents) 
features = (vectorizer.get_feature_names())
idf_scores = vectorizer.idf_

cnt_vct = CountVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
cnt_matrix = cnt_vct.fit_transform(word_documents)
cnt_features = cnt_vct.get_feature_names()

cnt_matrix_arr = cnt_matrix.toarray()
tfidf_new_matrix = cnt_matrix_arr*idf_scores

print(features)

['abat', 'abat methyl', 'abat methyl orang', 'abat methyl orang dye', 'absorpt', 'absorpt proceed', 'absorpt proceed fourth', 'absorpt proceed fourth intern', 'abus', 'abus languag', 'abus languag detect', 'abus languag detect dataset', 'acm', 'acm biometr', 'acm biometr base', 'acm biometr base design', 'acm mobihoc', 'acm mobihoc workshop', 'acm mobihoc workshop pervas', 'activ', 'activ bridg', 'activ bridg convert', 'activ bridg convert grid', 'activ line', 'activ line flow', 'activ line flow control', 'activ power', 'activ power loss', 'activ power loss minim', 'addit', 'addit free', 'addit free synthesi', 'addit free synthesi hematit', 'adopt', 'adopt ieee', 'adopt ieee electron', 'adopt ieee electron devic', 'adsorb', 'adsorb anion', 'adsorb anion dye', 'adsorb anion dye binari', 'adsorb methyl', 'adsorb methyl orang', 'adsorb methyl orang aqueou', 'adsorpt', 'adsorpt anion', 'adsorpt anion dye', 'adsorpt anion dye binari', 'adsorpt anion dye magnet', 'adsorpt cr', 'adsorpt cr vi

In [10]:
# print(tfidf_matrix)
# print(tfidf_matrix[(0, 3235)])

In [11]:
# print(author_term_citations)
# feature_idx = {}
# for idx, feature in enumerate(features):
#     feature_idx[feature] = idx
# author_idx = 0
# for author, author_dict in tqdm(author_term_citations.items(), leave=False):
#     for term, term_citations in author_dict.items():
#         if term in feature_idx:
#             term_idx = feature_idx[term]
#             log_factor = 1 + math.log( 1 + ((term_citations) / (term_total_citations[term] + 1)) , 2)
# #             print(log_factor)
#             tfidf_matrix[(author_idx, term_idx)] *= log_factor
#     author_idx += 1

In [12]:
# tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=0)

### Pickle the Matrix and Vectorizer

In [33]:
!mkdir -p web_data

In [34]:
pickle.dump(cnt_vct, open("web_data/count_vectorizer.pkl", "wb"))
pickle.dump(tfidf_new_matrix, open("web_data/tfidf_new_matrix.pkl", "wb"))
pickle.dump(docID, open("web_data/doc_id_dict.pkl", "wb"))

### Query Subjects

In [39]:
query="machine learning"
# print(features[3730])
query = process_string(query)
cnt_matrix_test = cnt_vct.transform([query])
cnt_matrix_test_arr = cnt_matrix_test.toarray()

res = np.matmul(tfidf_new_matrix,cnt_matrix_test_arr.T).squeeze()
related_docs_indices = res.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)

['owqaSZQAAAAJ', 'CmBE6P0AAAAJ', '1igRo6wAAAAJ', 'BaoAeqQAAAAJ', 'u_nJKdwAAAAJ', 'laT3mwoAAAAJ', 'Pp5WeBsAAAAJ', 'BJ7CzPgAAAAJ', 'eeHeiSAAAAAJ']


### Pickled Testing

In [24]:
cnt_vct = pickle.load(open("web_data/count_vectorizer.pkl", "rb"))
tfidf_new_matrix = pickle.load(open("web_data/tfidf_new_matrix.pkl", "rb"))
docID = pickle.load(open("web_data/doc_id_dict.pkl", "rb"))

In [25]:
query="data mining"
# print(features[3730])
query = process_string(query)
cnt_matrix_test = cnt_vct.transform([query])
cnt_matrix_test_arr = cnt_matrix_test.toarray()

res = np.matmul(tfidf_new_matrix,cnt_matrix_test_arr.T).squeeze()
related_docs_indices = res.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)

['laT3mwoAAAAJ', 'Pp5WeBsAAAAJ', 'BJ7CzPgAAAAJ', 'CmBE6P0AAAAJ', 'eeHeiSAAAAAJ', 'u_nJKdwAAAAJ', 'BaoAeqQAAAAJ', '1igRo6wAAAAJ', 'owqaSZQAAAAJ']
